# 1.3 Python data loading into SQLite database steps

In this notebook we'll load the final cleaned dataset `PPP-Data-up-to-150k-080820-HI-OpenRefine-PythonCleaned.csv` into a SQLite database.

We will use libraries like `pandas` for data manipulation and `sqlite3` for loading the data into the `PPP.db` SQLite database.

In [9]:
# Import necessary libraries
import sqlite3
import pandas as pd
import time

# Create a timer to time the execution of all steps
start = time.time()

### Step 1: Load the data

First, we’ll load the final cleaned dataset `PPP-Data-up-to-150k-080820-HI-OpenRefine-PythonCleaned.csv` into a pandas DataFrame.

An important detail is that we specify the `NAICSCode` column values as `string` type so Python correctly interprets the NAICS codes as categorical identifiers and not as numerical values.

In [10]:
# Step 1: Load the data with 'NAICSCode' specified as str
df = pd.read_csv('../../data/cleaned/PPP-Data-up-to-150k-080820-HI-OpenRefine-PythonCleaned.csv', dtype={'NAICSCode': str})
print(f"Step #1 -- Loaded the cleaned dataset, took {time.time() - start:.3f} seconds")

Step #1 -- Loaded the cleaned dataset, took 0.064 seconds


### Step 2: Create the connection to the SQLite database.

In [11]:
# If the SQLite database file does not exist, it will be created
conn = sqlite3.connect('../../data/database/PPP.db')

# Create the cursor
cur = conn.cursor()
print(f"Step #2 -- Created the connection to the `PPP.db` SQLite database, took {time.time() - start:.3f} seconds")

Step #2 -- Created the connection to the `PPP.db` SQLite database, took 0.071 seconds


### Step 3: Execute SQL commands to create the tables in the database.

In [12]:
# If the tables already exist, they are dropped and recreated
cur.execute('DROP TABLE IF EXISTS Addresses;')
cur.execute('''
CREATE TABLE IF NOT EXISTS Addresses (
    ID INTEGER PRIMARY KEY,
    City TEXT NOT NULL,
    State TEXT NOT NULL,
    Zip TEXT NOT NULL
);
''')
print(f"Created the `Addresses` table, took {time.time() - start:.3f} seconds")

cur.execute('DROP TABLE IF EXISTS Demographics;')
cur.execute('''
CREATE TABLE IF NOT EXISTS Demographics (
    ID INTEGER PRIMARY KEY,
    Gender TEXT NOT NULL,
    RaceEthnicity TEXT NOT NULL,
    Veteran TEXT NOT NULL
);
''')
print(f"Created the `Demographics` table, took {time.time() - start:.3f} seconds")

cur.execute('DROP TABLE IF EXISTS BusinessTypes;')
cur.execute('''
CREATE TABLE IF NOT EXISTS BusinessTypes (
    ID INTEGER PRIMARY KEY,
    BusinessType TEXT NOT NULL,
    NonProfit BOOLEAN NOT NULL
);
''')
print(f"Created the BusinessTypes table, took {time.time() - start:.3f} seconds")

cur.execute('DROP TABLE IF EXISTS Industries;')
cur.execute('''
CREATE TABLE IF NOT EXISTS Industries (
    ID INTEGER PRIMARY KEY,
    NAICSCode TEXT,
    NAICSTitle TEXT
);
''')
print(f"Created the `Industries` table, took {time.time() - start:.3f} seconds")

cur.execute('DROP TABLE IF EXISTS Loans;')
cur.execute('''
CREATE TABLE IF NOT EXISTS Loans (
    ID INTEGER PRIMARY KEY,
    LoanAmount REAL NOT NULL,
    JobsReported INTEGER,
    DateApproved TEXT NOT NULL,
    Lender TEXT NOT NULL,
    CD TEXT NOT NULL,
    AddressID INTEGER NOT NULL,
    DemographicID INTEGER NOT NULL,
    BusinessTypeID INTEGER NOT NULL,
    IndustryID INTEGER NOT NULL,
    FOREIGN KEY (AddressID) REFERENCES Addresses (ID),
    FOREIGN KEY (DemographicID) REFERENCES Demographics (ID),
    FOREIGN KEY (BusinessTypeID) REFERENCES BusinessTypes (ID),
    FOREIGN KEY (IndustryID) REFERENCES Industries (ID)
);
''')
print(f"Created the `Loans` table, took {time.time() - start:.3f} seconds")

# Commit the changes
conn.commit()
print(f"Step #3 -- Created all tables for the `PPP.db` SQLite database, took {time.time() - start:.3f} seconds")

Created the `Addresses` table, took 0.096 seconds
Created the `Demographics` table, took 0.109 seconds
Created the BusinessTypes table, took 0.125 seconds
Created the `Industries` table, took 0.138 seconds
Created the `Loans` table, took 0.154 seconds
Step #3 -- Created all tables for the `PPP.db` SQLite database, took 0.154 seconds


### Step 4: Split the original DataFrame into separate DataFrames for each table.

In [13]:
# Remove duplicates and reset the index to create a new 'ID' column.
# Then insert the data into the corresponding table in the SQLite database.

# Addresses table
addresses = df[['City', 'State', 'Zip']].drop_duplicates()
addresses.reset_index(inplace=True, drop=True)
addresses['ID'] = addresses.index + 1
addresses.to_sql('Addresses', conn, if_exists='append', index=False)
print(f"Inserted {len(addresses)} rows into the `Addresses` table, took {time.time() - start:.3f} seconds")

# Demographics table
demographics = df[['Gender', 'RaceEthnicity', 'Veteran']].drop_duplicates()
demographics.reset_index(inplace=True, drop=True)
demographics['ID'] = demographics.index + 1
demographics.to_sql('Demographics', conn, if_exists='append', index=False)
print(f"Inserted {len(demographics)} rows into the `Demographics` table, took {time.time() - start:.3f} seconds")

# BusinessTypes table
businesstypes = df[['BusinessType', 'NonProfit']].drop_duplicates()
businesstypes.reset_index(inplace=True, drop=True)
businesstypes['ID'] = businesstypes.index + 1
businesstypes.to_sql('BusinessTypes', conn, if_exists='append', index=False)
print(f"Inserted {len(businesstypes)} rows into the `BusinessTypes` table, took {time.time() - start:.3f} seconds")

# Industries table
industries = df[['NAICSCode', 'NAICSTitle']].drop_duplicates()
industries.reset_index(inplace=True, drop=True)
industries['ID'] = industries.index + 1
industries.to_sql('Industries', conn, if_exists='append', index=False)
print(f"Inserted {len(industries)} rows into the `Industries` table, took {time.time() - start:.3f} seconds")

print(f"Step #4 -- Inserted data into `Addresses`, `Demographics`, `BusinessTypes`, and `Industries` tables")

Inserted 119 rows into the `Addresses` table, took 0.177 seconds
Inserted 42 rows into the `Demographics` table, took 0.194 seconds
Inserted 15 rows into the `BusinessTypes` table, took 0.205 seconds
Inserted 756 rows into the `Industries` table, took 0.216 seconds
Step #4 -- Inserted data into `Addresses`, `Demographics`, `BusinessTypes`, and `Industries` tables


### Step 5: Merge on `key` to get `AddressID`, `DemographicID`, `BusinessTypeID` and `IndustryID` in the original dataframe

In [14]:
# Create a 'key' column in the original DataFrame and the other DataFrames
# This 'key' is a tuple of the columns being matched on
df['key'] = df[['City', 'State', 'Zip']].apply(tuple, axis=1)
addresses['key'] = addresses[['City', 'State', 'Zip']].apply(tuple, axis=1)

# Merge on 'key' to get 'AddressID'
df = df.merge(addresses[['key', 'ID']], on='key', how='left')

# Rename 'ID' to 'AddressID'
df.rename(columns={'ID': 'AddressID'}, inplace=True)

# Create a 'key' column in the Demographics df and merge on 'key' to get 'DemographicID'
df['key'] = df[['Gender', 'RaceEthnicity', 'Veteran']].apply(tuple, axis=1)
demographics['key'] = demographics[['Gender', 'RaceEthnicity', 'Veteran']].apply(tuple, axis=1)
df = df.merge(demographics[['key', 'ID']], on='key', how='left')
df.rename(columns={'ID': 'DemographicID'}, inplace=True)

# Create a 'key' column in the BusinessTypes df and merge on 'key' to get 'BusinessTypeID'
df['key'] = df[['BusinessType', 'NonProfit']].apply(tuple, axis=1)
businesstypes['key'] = businesstypes[['BusinessType', 'NonProfit']].apply(tuple, axis=1)
df = df.merge(businesstypes[['key', 'ID']], on='key', how='left')
df.rename(columns={'ID': 'BusinessTypeID'}, inplace=True)

# Create a 'key' column in the Industries df and merge on 'key' to get 'IndustryID'
df['key'] = df[['NAICSCode', 'NAICSTitle']].apply(tuple, axis=1)
industries['key'] = industries[['NAICSCode', 'NAICSTitle']].apply(tuple, axis=1)
df = df.merge(industries[['key', 'ID']], on='key', how='left')
df.rename(columns={'ID': 'IndustryID'}, inplace=True)

# Now that the merging is done, we can drop the 'key' column
df.drop(columns='key', inplace=True)

print(f"Step #5 -- Merged keys to get `AddressID`, `DemographicID`, `BusinessTypeID` and `IndustryID` "
      f"in cleaned dataframe, took {time.time() - start:.3f} seconds")

Step #5 -- Merged keys to get `AddressID`, `DemographicID`, `BusinessTypeID` and `IndustryID` in cleaned dataframe, took 0.666 seconds


### Step 6: Create a new `loans` DataFrame with the required columns and insert the data from it into the `Loans` table

In [15]:
# The index of the DataFrame is used as the 'ID' column in the Loans table
loans = df[
    ['LoanAmount', 'JobsReported', 'DateApproved', 'Lender', 'CD', 'AddressID', 'DemographicID', 'BusinessTypeID',
     'IndustryID']]
loans.reset_index(inplace=True, drop=True)
loans.index.name = 'ID'
loans.to_sql('Loans', conn, if_exists='append', index=True)

print(f"Step #6 -- Inserted data and foreign key values (`AddressID`, `DemographicID`, `BusinessTypeID`, `IndustryID`) "
      f"into the `Loans` table, took {time.time() - start:.3f} seconds")

Step #6 -- Inserted data and foreign key values (`AddressID`, `DemographicID`, `BusinessTypeID`, `IndustryID`) into the `Loans` table, took 0.727 seconds


### Step 7: Commit the changes and close the connection

In [16]:
conn.commit()
conn.close()
print(f"Step #7 -- Committed changes and closed the database connection")

Step #7 -- Committed changes and closed the database connection
